In [22]:
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import requests
import sqlite3

In [23]:
# server = "https://geo.api.vlaanderen.be/BWD-IdentifiedBathingWaters/ogc/features/collections/BWDBW/items?f=application%2Fgeo%2Bjson"
server = "https://geo.api.vlaanderen.be/VlaamseZwemwaterlocaties/ogc/features/collections/Vlzwl/items?f=application%2Fgeo%2Bjson"

In [24]:
# Azure App Gateway blocks Python requests lib by default
response = requests.get(server, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
})


In [26]:
df = pd\
    .json_normalize(response.json()['features'])\
    .drop(columns=['id', 'type', 'geometry_name', 'bbox', 'geometry.type', 'geometry.coordinates', 'properties.OIDN', 'properties.UIDN', 'properties.CYEAR', 'properties.NAMESPACE', 'properties.VERSIONID', 'properties.BEGINLIFE', 'properties.LOCALID', 'properties.ENDLIFE', 'properties.PREDECESID', 'properties.PREDEIDSCH', 'properties.SUCCESSOID', 'properties.SUCCEIDSCH', 'properties.WEVOLUTION', 'properties.NAMETXTLAN', 'properties.DESIGBEGIN', 'properties.DESIGEND', 'properties.ZONETYPE', 'properties.SPZONETYPE', 'properties.RZONEID', 'properties.RZONEIDSCH', 'properties.LEGISNAME', 'properties.LEGISLINK', 'properties.LEGISLEVEL', 'properties.THEMAIDSCH', 'properties.SIZEVALUE', 'properties.SIZEUOM', 'properties.CONFSTATUS', 'properties.LINKKWL', 'properties.STATUSCODE', 'properties.STATUSDATE', 'properties.REMARKS', 'properties.COUNTRY', 'properties.QCCHECK', 'properties.REMARKS2', 'properties.OPMERKINGE'])\
    .rename(columns={'properties.THEMATICID': 'id', 'properties.NAMETEXT': 'name', 'properties.NAMETXTINT': 'alternate_name', 'properties.LON': 'long', 'properties.LAT': 'lat'})\
    .set_index('id')
df['alternate_name'] = df['alternate_name'].str.title()
df['name'] = df['name'].str.title()
locations = df[['name', 'alternate_name', 'lat', 'long']]
locations

,name,alternate_name,lat,long
id,,,,
BEVL_BW_BLA02,Blankenberge Weststraat,Blankenberge Weststraat,51.316490,3.120356
BEVL_BW_BLA04,Blankenberge Kerkstraat,Blankenberge Kerkstraat,51.318140,3.127011
BEVL_BW_BLA06,Blankenberge Artanhelling,Blankenberge Artanhelling,51.320070,3.134678
BEVL_BW_BLE01,Berlare Nieuwdonk,Berlare Nieuwdonk,51.035760,3.970350
BEVL_BW_BRE01,Bredene Turkeyen,Bredene Turkeyen,51.248490,2.952251
...,...,...,...,...
BEVL_BW_MEC03,Mechelen Keerdok,Mechelen Keerdok,51.032870,4.470210
BEVL_BW_IEP01,Ieper Dikkebusvijver,Ieper Dikkebusvijver,50.821564,2.847248
BEVL_BW_LIR01,Lier Anderstad,Lier Anderstad,51.114790,4.548050


In [27]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")


99